In [1]:
from langchain_community.document_loaders import PyPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [2]:
# Read the pdf from the folder
loader = PyPDFDirectoryLoader("./us_census")

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)

final_document = text_splitter.split_documents(documents)
final_document[0]

Document(metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under \nthe Continuous Enrollment Provision.2 The American \nRescue Plan (ARP) enhanced Marketplace premium \nsubsidies for those with incomes above 400 percent \nof the poverty level as well as for unemployed people.3\nIn addition to national policies, individual states and \nthe District of Columbia can affect 

In [3]:
# Embedding Using Huggingface
huggingface_embedding = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5" ,
    model_kwargs={'device' : 'cpu'},
    encode_kwargs={'normalizer_embedding' : True}
)

d:\gen-ai\langcain-tut\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\gen-ai\langcain-tut\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Kartikeya\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode,

In [4]:
import numpy as np
np.array(huggingface_embedding.embed_query(final_document[0].page_content))

array([-0.07903484, -0.01134112, -0.02312097,  0.0284446 ,  0.05053344,
        0.05317826, -0.01907787,  0.03456026, -0.10211367, -0.02915701,
        0.08524259,  0.05650727, -0.02545438, -0.0330849 , -0.00635735,
        0.04090863, -0.00628108,  0.00356744, -0.03854128,  0.03667685,
       -0.04289803,  0.03425251, -0.03116899, -0.03793729,  0.01728391,
        0.01214924,  0.00653119,  0.01463565, -0.05529054, -0.15320711,
        0.00730845,  0.03202944, -0.04701132, -0.01595974,  0.0187445 ,
        0.02642936, -0.02306378,  0.08438034,  0.04182484,  0.05278176,
       -0.03057602,  0.01564262, -0.01689073,  0.00529409, -0.02417435,
        0.00412995, -0.01889937, -0.00150625, -0.00836944, -0.03390064,
        0.0351596 , -0.00553131,  0.04910937,  0.05971855,  0.05615962,
       -0.05105155,  0.01475136, -0.01849959, -0.03284641,  0.03576624,
        0.04947704, -0.00938883, -0.26202115,  0.09750329,  0.01715692,
        0.0478139 , -0.00556317, -0.00298307, -0.02207355, -0.04

In [5]:
## VectorStore Creation
vectorstore=FAISS.from_documents(final_document[:120],huggingface_embedding)

In [7]:
# Query using similarity Search
query = 'WHAT IS HEALTH INSURANCE COVERAGE?'

relevant_document = vectorstore.similarity_search(query)
relevant_document[0].page_content

'2 U.S. Census Bureau\nWHAT IS HEALTH INSURANCE COVERAGE?\nThis brief presents state-level estimates of health insurance coverage \nusing data from the American Community Survey (ACS). The  \nU.S. Census Bureau conducts the ACS throughout the year; the \nsurvey asks respondents to report their coverage at the time of \ninterview. The resulting measure of health insurance coverage, \ntherefore, reflects an annual average of current comprehensive \nhealth insurance coverage status.* This uninsured rate measures a \ndifferent concept than the measure based on the Current Population \nSurvey Annual Social and Economic Supplement (CPS ASEC). \nFor reporting purposes, the ACS broadly classifies health insurance \ncoverage as private insurance or public insurance. The ACS defines \nprivate health insurance as a plan provided through an employer \nor a union, coverage purchased directly by an individual from an \ninsurance company or through an exchange (such as healthcare.'

In [12]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ['HUGGINGFACEHUB_API_TOKEN']= os.environ.get('HUGGINGFACEHUB_API_TOKEN')

In [8]:
retriever = vectorstore.as_retriever(search_type = 'similarity',search_kwargs ={'k':3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000284C4FFE650> search_kwargs={'k': 3}


In [18]:
from langchain_community.llms import HuggingFaceHub
hf = HuggingFaceHub(
    repo_id = 'mistralai/Mistral-7B-v0.1',
    model_kwargs = {'temprature':0.1,"max_lenght":500}
)
query = "what is the health insurance coverage?"
hf.invoke(query)

'what is the health insurance coverage?\n\nThe health insurance coverage is a type of insurance that covers the cost of medical care. It is usually provided by an employer, but can also be purchased individually. The coverage typically includes hospitalization, doctor visits, and prescription drugs.\n\nWhat are the benefits of health insurance coverage?\n\nThe benefits of health insurance coverage are many. It can help you pay for medical expenses, protect you from financial ruin in the event of a serious illness or injury, and provide peace of mind knowing'

### Loading The model locally
This is for running model locally so You can uncomment this to run the model locally

In [17]:
# Huggingface models can be run locally through the HuggingFacePipeline class
# from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# hf = HuggingFacePipeline.from_model_id(
#     model_id = 'mistralai/Mistral-7B-v0.1',
#     task='text-generation',
#     pipeline_kwargs = {'temperature':0,'max_new_tokens':300}
# )

# llm = hf
# llm.invoke(query)

In [20]:
prompt_template = """
User the following piece of context to answer the question asked.
Please try to provide the answer only based on the context
{context}
Question:{question}
Helpful Answers:
"""

In [21]:
prompt = PromptTemplate(template=prompt_template,input_variables=["context",'question'])

In [28]:
retrieverQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [30]:
query = """Differences in the uninsured Rate by state is 2022"""

In [31]:
# Call the QA chain with our query
result = retrieverQA.invoke({'query':query})
result['result']

'\nUser the following piece of context to answer the question asked.\nPlease try to provide the answer only based on the context\npercent (Appendix Table B-5). \nMedicaid coverage accounted \nfor a portion of that difference. \nMedicaid coverage was 22.7 per-\ncent in the group of states that \nexpanded Medicaid eligibility and \n18.0 percent in the group of nonex-\npansion states.\nCHANGES IN THE UNINSURED \nRATE BY STATE FROM 2021 \nTO 2022\nFrom 2021 to 2022, uninsured rates \ndecreased across 27 states, while \nonly Maine had an increase. The \nuninsured rate in Maine increased \nfrom 5.7 percent to 6.6 percent, \nalthough it remained below the \nnational average. Maine’s uninsured \nrate was still below 8.0 percent, \n21 Douglas Conway and Breauna Branch, \n“Health Insurance Coverage Status and Type \nby Geography: 2019 and 2021,” 2022, <www.\ncensus.gov/content/dam/Census/library/\npublications/2022/acs/acsbr-013.pdf>.\n\ncomparison of ACS and CPS ASEC measures \nof health insura